<a href="https://colab.research.google.com/github/KingAbdulRehman/LangChain/blob/main/LangChain_ToolCalling/LangChain_Calculator_Tool_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instruction:

Setup Gemeni Api key as **"GOOGLE_API_KEY"** in Google Colab Secret


# Install and Import Packages

In [ ]:
!pip install langchain-google-genai

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from google.colab import userdata
from langchain_core.tools import tool
from langchain import hub
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.agents.schema import AgentScratchPadChatPromptTemplate
from rich.console import Console
from rich.markdown import Markdown
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Setup API Keys

In [4]:
if "GOOGLE_API_KEY" not in os.environ:
  os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Setup LLM

In [5]:
llm = ChatGoogleGenerativeAI(
  model="gemini-2.0-flash-exp",
  max_tokens=1000,
  max_retries=3,
  top_k=2,
  temperature=0.7
)

In [6]:
# test llm
print(llm.invoke("Hello").content)

Hello! How can I help you today?


# Create Search Tool

In [7]:
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            # Use Python's eval to compute the result safely
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)
        except Exception as e:
            return f"Error: {e}"


@tool
def calculator(expression: str) -> str:
    """
    Perform arithmetic calculations.
    Input: A mathematical expression as a string (e.g., "2 + 2").
    Output: Result of the calculation as a string.
    """
    calc = Calculator()
    return calc.calculate(expression)

In [8]:
# test tool
calculator("(2 + 2)-(6 + 4)")

<ipython-input-8-42b07fd2e8a3>:2: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  calculator("(2 + 2)-(6 + 4)")


'-6'

# Create Tool Calling Agent

In [10]:
tools = [calculator]

prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_tool_calling_agent(
  tools=tools,
  llm=llm,
  prompt=prompt
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [13]:
ai_search_res = agent_executor.invoke({"input": "Please solve: 2 * 5 + 7 / 18"})



> Entering new AgentExecutor chain...

Invoking: `calculator` with `{'expression': '2 * 5 + 7 / 18'}`


10.38888888888889The answer is 10.388888888888889.

> Finished chain.


In [14]:
console = Console()
console.print(Markdown(ai_search_res["output"]))

The answer is 10.388888888888889.

# Add Chat history

In [15]:
memory = InMemoryChatMessageHistory(session_id="test-session")

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="output"  # Ensure this key matches the output of your agent
)

config = {"configurable": {"session_id": "test-session"}}

In [16]:
print(
    agent_with_chat_history.invoke(
        {"input": "Please solve 10 * 20 * 100 / 150"}, config
    )["output"]
)



> Entering new AgentExecutor chain...

Invoking: `calculator` with `{'expression': '10 * 20 * 100 / 150'}`


133.3333333333333410 * 20 * 100 / 150 = 133.33333333333334

> Finished chain.
10 * 20 * 100 / 150 = 133.33333333333334


In [17]:
print(
    agent_with_chat_history.invoke(
        {"input": "did you tell me what i ask previosuly?"}, config
    )["output"]
)



> Entering new AgentExecutor chain...
Yes, I did. I calculated the result of the expression 10 * 20 * 100 / 150, which is 133.33333333333334.

> Finished chain.
Yes, I did. I calculated the result of the expression 10 * 20 * 100 / 150, which is 133.33333333333334.
